# Lending Club Data

## Data Analysis

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from custom_log_regression import CustomLogRegression

DATA_PATH = './data/DR_Demo_Lending_Club_reduced.csv'
RND_STATE = 42

In [2]:
numeric_cols = ['annual_inc', 'collections_12_mths_ex_med', 'debt_to_income',
    'delinq_2yrs', 'revol_util', 'emp_length', 'total_acc', 'inq_last_6mths',
    'mths_since_last_major_derog', 'open_acc', 'pub_rec', 'revol_bal']

categorical_cols = ['addr_state', 'home_ownership', 'initial_list_status', 'zip_code', 'pymnt_plan',
    'policy_code', 'verification_status', 'purpose_cat']

In [31]:
# which categorical variables do we have and there values
import numpy as np

for cat in categorical_cols:
    unique_values = np.unique(data[cat].values)
    print(f'{cat} values: {unique_values}')

addr_state values: ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD' 'TN'
 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY']


TypeError: '<' not supported between instances of 'str' and 'float'

In [4]:
# map selected categorical variables to numerical values
def verification_status_mapper(vstatus):
    if 'VERIFIED' in vstatus:
        return 1
    return 0

def pymnt_plan_mapper(pymnt_plan):
    if pymnt_plan == 'y':
        return 1
    return 0

def initial_list_status_mapper(initial_list_status):
    if initial_list_status == 'm':
        return 1
    return 0

home_ownership_codes = {
        'NONE': None,
        'OTHER': 0,
        'MORTGAGE': 1,
        'OWN': 2,
        'RENT': 3
    }
    
data.verification_status = data.verification_status.map(verification_status_mapper)
data.pymnt_plan = data.pymnt_plan.map(pymnt_plan_mapper)
data.initial_list_status = data.initial_list_status.map(initial_list_status_mapper)
data.policy_code = data.policy_code.map(lambda p: int(p[-1]))
data.home_ownership = data.home_ownership.map(lambda c: home_ownership_codes[c])

In [6]:
drop_cols = ['addr_state', 'zip_code', 'purpose_cat', 
             'mths_since_last_delinq', 'mths_since_last_record']
data = data.drop(drop_cols, axis=1)

## Train Classifier

In [2]:
# clean non-numeric values in numeric columns
# not correctly placed

test_size = 0.3

clf = CustomLogRegression(random_state=RND_STATE, n_jobs=-1)
X_train, X_test, y_train, y_test = clf.read_data(data_path=DATA_PATH, y_col='is_bad', index_col='Id',
                                                 skip_cols=['zip_code', 'addr_state'])
params = clf.tune_parameters(X_train, y_train)
print(params)

clf.fit(X_train, y_train)
metrics = clf.evaluate(X_test, y_test)

print('Accuracy {:.2}, F1 Score {:.2}, Logistic Loss {:.2}'.format(
    *metrics.values()
))

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sk

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/prepr

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/prepr

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/prepr

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sk

...

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/prepr

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/prepr

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/base.py:467: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/pipeline.py:511: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/prepr

{'C': 0.0008111308307896872, 'scores': {'accuracy': array([0.60647268, 0.60647268, 0.60647268, 0.60647268, 0.60647268,
       0.60647268, 0.60647268, 0.60662006, 0.6069144 , 0.60706167,
       0.60706167, 0.60706167, 0.76541306, 0.76511851, 0.76467657,
       0.80367583, 0.80338128, 0.80279207, 0.80235003, 0.80161333,
       0.80131845, 0.80072935, 0.80043491, 0.79969853, 0.79896226,
       0.79852022, 0.79763646, 0.79542711, 0.79395403, 0.79248116,
       0.78938903, 0.78585344, 0.78334911, 0.77922496, 0.77642608,
       0.77318613, 0.76817908, 0.76125844, 0.75595652, 0.74888773,
       0.74270205, 0.73828271, 0.73563077, 0.73150695, 0.72738356,
       0.72340745, 0.71943156, 0.71619182, 0.7139828 , 0.71133216,
       0.70853371, 0.70720737, 0.70529257, 0.70396709, 0.70234663,
       0.70102104, 0.69925395, 0.69778087, 0.69748621, 0.69630801,
       0.69483569, 0.69468841, 0.69454135, 0.69336315, 0.6936577 ,
       0.69351053, 0.69351053, 0.69365792, 0.69336358, 0.69351097,
       0.6

/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/Projects/home-test-dr/custom_log_regression.py:98: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  X_scaled = scaler.transform(X)
/home/okozyn/anaconda3/envs/mne/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/okozyn/Projects/home-test-dr/custom_log_regression.py:159: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  X_scaled = scaler.transform(X)


In [7]:
X_train.columns

Index(['pub_rec', 'revol_util', 'mths_since_last_delinq', 'policy_code_PC4',
       'policy_code_PC1', 'policy_code_PC2', 'policy_code_PC3',
       'policy_code_PC5', 'mths_since_last_major_derog', 'total_acc',
       ...
       'addr_state_ID', 'addr_state_UT', 'addr_state_MT', 'addr_state_WY',
       'addr_state_VT', 'addr_state_TN', 'addr_state_IA', 'addr_state_MS',
       'addr_state_NE', 'collections_12_mths_ex_med'],
      dtype='object', length=818)

In [9]:
data = pd.read_csv(DATA_PATH, index_col='Id', na_values=['NONE', 'na'])
data.isna().sum()

is_bad                            0
emp_length                      250
home_ownership                    1
annual_inc                        1
verification_status               0
pymnt_plan                        0
purpose_cat                       0
zip_code                          0
addr_state                        0
debt_to_income                    0
delinq_2yrs                       5
inq_last_6mths                    5
mths_since_last_delinq         6316
mths_since_last_record         9160
open_acc                          5
pub_rec                           5
revol_bal                         0
revol_util                       26
total_acc                         5
initial_list_status               0
collections_12_mths_ex_med       32
mths_since_last_major_derog       0
policy_code                       0
dtype: int64

In [4]:
X_test.shape

(2910, 55)